In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
#part 1 :preprocessing
import pandas as pd

# Loading the dataset
df = pd.read_csv('/kaggle/input/movie-review/movie_review.csv')


df

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [18]:
# Cleaning
import re

# Clean the 'text' column: removing punctuation and underscores
df["text"] = df["text"].apply(lambda x: re.sub(r'[^\w\s]|_', '', str(x)))


print(df)

       fold_id cv_tag  html_id  sent_id  \
0            0  cv000    29590        0   
1            0  cv000    29590        1   
2            0  cv000    29590        2   
3            0  cv000    29590        3   
4            0  cv000    29590        4   
...        ...    ...      ...      ...   
64715        9  cv999    14636       20   
64716        9  cv999    14636       21   
64717        9  cv999    14636       22   
64718        9  cv999    14636       23   
64719        9  cv999    14636       24   

                                                    text  tag  
0      films adapted from comic books have had plenty...  pos  
1      for starters  it was created by alan moore  an...  pos  
2      to say moore and campbell thoroughly researche...  pos  
3      the book  or  graphic novel   if you will  is ...  pos  
4      in other words  dont dismiss this film because...  pos  
...                                                  ...  ...  
64715  that lack of inspiration can

In [19]:
#Normalize the text  column:

df["text"] = df["text"].str.lower()
print(df)

       fold_id cv_tag  html_id  sent_id  \
0            0  cv000    29590        0   
1            0  cv000    29590        1   
2            0  cv000    29590        2   
3            0  cv000    29590        3   
4            0  cv000    29590        4   
...        ...    ...      ...      ...   
64715        9  cv999    14636       20   
64716        9  cv999    14636       21   
64717        9  cv999    14636       22   
64718        9  cv999    14636       23   
64719        9  cv999    14636       24   

                                                    text  tag  
0      films adapted from comic books have had plenty...  pos  
1      for starters  it was created by alan moore  an...  pos  
2      to say moore and campbell thoroughly researche...  pos  
3      the book  or  graphic novel   if you will  is ...  pos  
4      in other words  dont dismiss this film because...  pos  
...                                                  ...  ...  
64715  that lack of inspiration can

In [22]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# Tokenization: splitting the string into a list of words
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
#used a lambda expression :
df['text'] = df['text'].apply(lambda x: word_tokenize(str(x)))

print(df['text'])

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
0        [films, adapted, from, comic, books, have, had...
1        [for, starters, it, was, created, by, alan, mo...
2        [to, say, moore, and, campbell, thoroughly, re...
3        [the, book, or, graphic, novel, if, you, will,...
4        [in, other, words, dont, dismiss, this, film, ...
                               ...                        
64715    [that, lack, of, inspiration, can, be, traced,...
64716    [like, too, many, of, the, skits, on, the, cur...
64717    [after, watching, one, of, the, roxbury, skits...
64718         [bump, unsuspecting, women, and, thats, all]
64719    [after, watching, anightattheroxbury, youll, b...
Name: text, Length: 64720, dtype: object


In [23]:
#Removing stop_words : Removing not meaningful words.
from nltk.corpus import stopwords

nltk.download('stopwords')

# Get the English stop words

stop_words = stopwords.words('english')


df['text'] = df['text'].apply(lambda sentence_tokens: [word for word in sentence_tokens if word not in stop_words ])

print(df['text'])

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
0        [films, adapted, comic, books, plenty, success...
1        [starters, created, alan, moore, eddie, campbe...
2        [say, moore, campbell, thoroughly, researched,...
3        [book, graphic, novel, 500, pages, long, inclu...
4                     [words, dont, dismiss, film, source]
                               ...                        
64715    [lack, inspiration, traced, back, insipid, cha...
64716    [like, many, skits, current, incarnation, satu...
64717    [watching, one, roxbury, skits, snl, come, awa...
64718                   [bump, unsuspecting, women, thats]
64719    [watching, anightattheroxbury, youll, left, ex...
Name: text, Length: 64720, dtype: object


In [25]:
#word2vec :
import gensim
#affecting to each word of my dataSet a vector (size=100)

# training my Word2Vec model
model =gensim.models.Word2Vec(sentences=df['text'], vector_size=100, window=5, min_count=1, workers=8)


In [29]:
#we can use the model to calculate the similariy between two words
model.wv.similarity(w1="films", w2="comic")

0.709806

In [28]:
# for example the vector representing the word films is : 
word_vector = model.wv['films']
word_vector 

array([-6.59076035e-01,  1.19481528e+00,  6.67670965e-01,  8.76661956e-01,
        1.47170734e+00, -7.90645182e-01,  1.43346381e+00,  1.94002759e+00,
       -6.87446177e-01,  4.28435877e-02, -1.04348734e-01, -1.92373025e+00,
        7.52190471e-01,  4.07082736e-01,  4.34483200e-01, -5.68841696e-01,
        6.39031172e-01,  1.68243289e-01, -6.34219885e-01, -2.10728884e+00,
        7.33670294e-01,  2.51250684e-01,  7.61865854e-01, -1.77564991e+00,
        1.20582774e-01, -5.99515378e-01, -1.13715804e+00, -4.02291983e-01,
       -1.27393126e+00,  3.73971090e-02,  1.31265700e-01, -2.06803232e-01,
        1.07967782e+00,  1.49195374e-03, -1.19922245e+00,  1.38112044e+00,
        5.25577068e-01,  1.00421570e-01,  9.64614600e-02, -1.37798333e+00,
       -8.83434936e-02, -4.82830495e-01,  1.49303436e-01,  4.42844927e-01,
        8.43595624e-01,  3.70592654e-01, -1.96873605e-01,  1.69483215e-01,
       -2.11658068e-02,  7.57974386e-01,  5.47954559e-01,  3.22361946e-01,
        4.73044008e-01, -

In [31]:
import numpy as np
# pour chaque mot du vocabulaire , on va récuperer le vecteur correspondants et on va  stocké l'ensemble de ces vecteurs dans un tab
def get_word_embeddings(review, model):
    word_embeddings = []
    for word in review:
        #Vérifier Si le mot est présent dans le vocabulaire du modèle :c'est-à-dire qu'il a été vu lors de l'entraînement du modèle
        if word in model.wv.key_to_index:
            word_embeddings.append(model.wv[word])
    return word_embeddings

# Fonction pour obtenir le vecteur moyen d'une critique
def get_review_vector(review, model):
    word_embeddings = get_word_embeddings(review, model)
    if word_embeddings:
        return np.mean(word_embeddings, axis=0)
    else:
        # Si la critique ne contient pas de mots dans le vocabulaire du modèle, retourner un vecteur nul
        return np.zeros(model.vector_size)


df['review_vector'] = df['text'].apply(lambda x: get_review_vector(x, model))
df['review_vector']

0        [-0.29437578, 0.6844872, 0.27564168, 0.0929046...
1        [-0.16242625, 0.39859912, 0.23955342, 0.134385...
2        [-0.30801386, 0.7278831, 0.26668298, 0.0373448...
3        [-0.17460036, 0.4626064, 0.25689456, 0.1374812...
4        [-0.3734592, 0.79059666, 0.2860926, 0.14194557...
                               ...                        
64715    [-0.118355155, 0.6174714, 0.26097655, 0.001273...
64716    [-0.21660297, 0.60104614, 0.22306864, 0.031008...
64717    [-0.32119262, 0.7532886, 0.3251984, 0.13015431...
64718    [-0.24686356, 0.6078703, 0.24496786, 0.0766055...
64719    [-0.47334653, 0.7256668, 0.2553075, 0.23306403...
Name: review_vector, Length: 64720, dtype: object

In [38]:
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split

# Convertir les listes de vecteurs en matrices NumPy : pour que les features et tragets aient des formes compatibles
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

# Diviser les données : partie training te testing
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

# Créer et entraîner le modèle de régression logistique
model1 = linear_model.LogisticRegression()
model1.fit(X_train, y_train)

# Évaluer le modèle sur l'ensemble de test
accuracy = model1.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.5585168018539977


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [39]:
#prediction :(sachant que le .score l'effectue implicitement)
y_pred = model1.predict(X_test)


print("Prédictions :", y_pred)

Prédictions : ['pos' 'neg' 'neg' ... 'pos' 'neg' 'neg']


In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculer les prédictions sur l'ensemble de test
y_pred = model1.predict(X_test)

# Calculer les métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Afficher les résultats
print("Accuracy :", accuracy)
print("Precision :", precision)
print("Recall :", recall)
print("F1-score :", f1)


Accuracy : 0.5585168018539977
Precision : 0.5592600732320318
Recall : 0.5585168018539977
F1-score : 0.5530040369605221
